In [ ]:
%run generator/statementGenerator.ipynb
%run generator/IDsPlayerTimeGenerator.ipynb
%run generator/TypeChecker.ipynb

**Select number of traces, players and IDs** to generate

In [ ]:
nbtracesAlternative = widgets.IntSlider(description="Traces", min=0, value=1, max=10000, continuous_update=False)
nbtracesAlternativetext = widgets.IntText(description="", continuous_update=False)
widgets.link((nbtracesAlternative, 'value'), (nbtracesAlternativetext, 'value'))
tracesAlternative=VBox([nbtracesAlternative,nbtracesAlternativetext])

nbplayersAlternative = widgets.IntSlider(description="Players", min=0, value=1, continuous_update=False)
nbplayersAlternativetext = widgets.IntText(description="", continuous_update=False)
widgets.link((nbplayersAlternative, 'value'), (nbplayersAlternativetext, 'value'))
widgets.link((nbtracesAlternative, 'value'), (nbplayersAlternative, 'max'))
playersAlternative=VBox([nbplayersAlternative,nbplayersAlternativetext])

nbIDsAlternative = widgets.IntSlider(description="IDs", min=0, value=1, continuous_update=False)
nbIDsAlternativetext = widgets.IntText(description="", continuous_update=False)
widgets.link((nbIDsAlternative, 'value'), (nbIDsAlternativetext, 'value'))
widgets.link((nbtracesAlternative, 'value'),(nbIDsAlternative, 'max'))
IDsAlternative=VBox([nbIDsAlternative,nbIDsAlternativetext])

vbox_nb_traces_ID_player_alternative=VBox([tracesAlternative,playersAlternative, IDsAlternative])

**Alternatives**

In [ ]:
verbs_alternatives=["selected", "unlocked"]
checkboxes_verbs_alternatives=GeneratorMultiCheckboxWidget("Aternatives verbs", verbs_alternatives)

types_alternatives=["question", "menu", "dialog", "path", "arena", "alternative"]
checkboxes_types_alternatives=GeneratorMultiCheckboxWidget("Aternatives types", types_alternatives)

template_result_alternative_response={
	"response": None,
	"success": None
}

def generate_alternatives(change=None):
    start_time=datetime.now()
    if change!=None:
        with output:
            print("Starting the data generation")
    verbs=checkboxes_verbs_alternatives.get_selected_options()
    types=checkboxes_types_alternatives.get_selected_options()
    ids=generate_IDs(nbIDsAlternative.value)
    times=generate_times_datas(nbtracesAlternative.value)
    player=generate_players(nbplayersAlternative.value)
    statements=[]
    checkTypes={}
    checkAnswer={}
    for i in range(nbtracesAlternative.value):
        randomID=random.choice(ids)
        randomPlayer=random.choice(player)
        randomVerb=random.choice(verbs)
        randomType = checkType(checkTypes,randomID,randomPlayer,types)
        time=times[i]
        statement=generator(randomPlayer, randomVerb, randomType,randomID, time)
        if randomType not in checkAnswer.keys():
            checkAnswer[randomType]={}
        if randomVerb=="selected":
            if randomID not in checkAnswer[randomType].keys():
                checkAnswer[randomType][randomID]=[]
                resultTrue=copy.deepcopy(template_result_alternative_response)
                resultTrue["response"]=str(uuid.uuid4())
                resultTrue["success"]="True"
                checkAnswer[randomType][randomID].append(resultTrue)
                checkAnswer[randomType][randomID].append("custom")
            result=random.choice(checkAnswer[randomType][randomID])
            if result=="custom":
                result=copy.deepcopy(template_result_alternative_response)
                result["response"]=str(uuid.uuid4())
                result["success"]="False"
                checkAnswer[randomType][randomID].append(result)
            if randomType in ["alternative","question"]:
                statement["result"]=result
            else: #menu dialog path arena
                statement["result"]={"response":result["response"]}
        statements.append(statement)
    if change is None :
        return statements
    else:
        save_generated_data(statements, verbs, types)
        end_time=datetime.now()
        delta=end_time-start_time
        with output:
            print("End the data generation")
            print("Generated in "+str(delta))
            clear_output(wait=True)

buttonAlternative=widgets.Button(description='Generate Alternative Data', layout = widgets.Layout( width='auto'))
buttonAlternative.on_click(generate_alternatives)

global tabAlternatives
tabAlternatives=VBox([
    HBox([
        checkboxes_verbs_alternatives.multi_select,
        checkboxes_types_alternatives.multi_select,
        vbox_nb_traces_ID_player_alternative
    ]),
    buttonAlternative
])